In [1]:
import numpy as np
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sn

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [2]:
covid_df = pd.read_csv('../optional_datasets/COVID-19 Reported Patient Impact and Hospital Capacity by Facility')
covid_df

,hospital_pk,collection_week,state,ccn,hospital_name,address,city,zip,hospital_subtype,fips_code,...,previous_day_admission_adult_covid_suspected_30-39_7_day_sum,previous_day_admission_adult_covid_suspected_40-49_7_day_sum,previous_day_admission_adult_covid_suspected_50-59_7_day_sum,previous_day_admission_adult_covid_suspected_60-69_7_day_sum,previous_day_admission_adult_covid_suspected_70-79_7_day_sum,previous_day_admission_adult_covid_suspected_80+_7_day_sum,previous_day_admission_adult_covid_suspected_unknown_7_day_sum,previous_day_admission_pediatric_covid_suspected_7_day_sum,previous_day_total_ED_visits_7_day_sum,previous_day_admission_influenza_confirmed_7_day_sum
0,3b081d5ef1c552538e4af4aa593a857bb922a4f364a412...,2021-02-26,LA,NaN,Surgery Center of Zachary,NaN,NaN,70791.0,Short Term,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,976b4ce7c95e1cd394112ce7762ed040f18a141e56a52e...,2021-02-26,PR,NaN,Hospital Industrial C.F.S.E,Paseo Dr. Jose Celso Barbosa,San Juan,935.0,Short Term,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ee04edd185865c38c839812cb2eb5ae5d3f8922e3b629e...,2021-02-26,LA,NaN,Alexandria Emergency Hospital,5900 Coliseum Blvd,Alexandria,71303.0,Short Term,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,aab2bb3ab769da90baf57242c96ec481afb5ec6a233784...,2021-02-26,LA,NaN,Crescent City Surgical Centre,NaN,NaN,70118.0,Short Term,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,452122,2021-02-26,TX,452122,El Paso LTAC Hospital,NaN,NaN,79902.0,Long Term,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154263,400130,2020-07-31,PR,400130,HOSPITAL METROPOLITANO DE LA MONTANA,CALLE ISAAC GONZALEZ ESQUINA LEDESMA,BDA NUEVA,641.0,Short Term,72141.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0,NaN
154264,400115,2020-07-31,PR,400115,CENTRO MEDICO WILMA N VAZQUEZ,CARR. 2 KM 39.5 ROAD NUMBER 2 BO ALGARROBO,VEGA BAJA,693.0,Short Term,72145.0,...,4.0,20.0,9.0,12.0,9.0,10.0,0.0,4.0,170.0,NaN
154265,400110,2020-07-31,PR,400110,HOSPITAL METROPOLITANO DR TITO MATTEI,ROAD 128 KM 1.0,YAUCO,698.0,Short Term,72153.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,274.0,NaN
154266,480002,2020-07-31,VI,480002,GOV JUAN F LUIS HOSPITAL & MEDICAL CTR,"#4007 EST DIAMOND RUBY, CHRISTIANSTED",ST CROIX,820.0,Short Term,78010.0,...,0.0,0.0,0.0,0.0,0.0,-999999.0,0.0,0.0,193.0,NaN


In [3]:
covid_df.columns

Index(['hospital_pk', 'collection_week', 'state', 'ccn', 'hospital_name',
       'address', 'city', 'zip', 'hospital_subtype', 'fips_code',
       'is_metro_micro', 'total_beds_7_day_avg',
       'all_adult_hospital_beds_7_day_avg',
       'all_adult_hospital_inpatient_beds_7_day_avg',
       'inpatient_beds_used_7_day_avg',
       'all_adult_hospital_inpatient_bed_occupied_7_day_avg',
       'total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg',
       'total_adult_patients_hospitalized_confirmed_covid_7_day_avg',
       'total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg',
       'total_pediatric_patients_hospitalized_confirmed_covid_7_day_avg',
       'inpatient_beds_7_day_avg', 'total_icu_beds_7_day_avg',
       'total_staffed_adult_icu_beds_7_day_avg', 'icu_beds_used_7_day_avg',
       'staffed_adult_icu_bed_occupancy_7_day_avg',
       'staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_avg',
       'staffed_icu_adult_p

In [4]:
covid_df.isna().sum()

hospital_pk                                                           0
collection_week                                                       0
state                                                                 0
ccn                                                                 500
hospital_name                                                         0
                                                                  ...  
previous_day_admission_adult_covid_suspected_80+_7_day_sum         9149
previous_day_admission_adult_covid_suspected_unknown_7_day_sum     7733
previous_day_admission_pediatric_covid_suspected_7_day_sum         6151
previous_day_total_ED_visits_7_day_sum                             4476
previous_day_admission_influenza_confirmed_7_day_sum              57698
Length: 93, dtype: int64

In [5]:
#drop null values
covid_df.dropna(how='any', axis=0, inplace=True)
covid_df

,hospital_pk,collection_week,state,ccn,hospital_name,address,city,zip,hospital_subtype,fips_code,...,previous_day_admission_adult_covid_suspected_30-39_7_day_sum,previous_day_admission_adult_covid_suspected_40-49_7_day_sum,previous_day_admission_adult_covid_suspected_50-59_7_day_sum,previous_day_admission_adult_covid_suspected_60-69_7_day_sum,previous_day_admission_adult_covid_suspected_70-79_7_day_sum,previous_day_admission_adult_covid_suspected_80+_7_day_sum,previous_day_admission_adult_covid_suspected_unknown_7_day_sum,previous_day_admission_pediatric_covid_suspected_7_day_sum,previous_day_total_ED_visits_7_day_sum,previous_day_admission_influenza_confirmed_7_day_sum
10,010108,2021-02-26,AL,010108,PRATTVILLE BAPTIST HOSPITAL,124 S MEMORIAL DR,PRATTVILLE,36067.0,Short Term,1001.0,...,0.0,0.0,-999999.0,-999999.0,-999999.0,0.0,0.0,0.0,478.0,0.0
11,010100,2021-02-26,AL,010100,THOMAS HOSPITAL,750 MORPHY AVENUE,FAIRHOPE,36532.0,Short Term,1003.0,...,0.0,0.0,-999999.0,-999999.0,0.0,-999999.0,0.0,0.0,837.0,0.0
12,010129,2021-02-26,AL,010129,NORTH BALDWIN INFIRMARY,1815 HAND AVENUE,BAY MINETTE,36507.0,Short Term,1003.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-999999.0,265.0,0.0
13,010083,2021-02-26,AL,010083,SOUTH BALDWIN REGIONAL MEDICAL CENTER,1613 NORTH MCKENZIE STREET,FOLEY,36535.0,Short Term,1003.0,...,0.0,-999999.0,-999999.0,22.0,16.0,12.0,0.0,0.0,627.0,0.0
14,010069,2021-02-26,AL,010069,MEDICAL CENTER BARBOUR,820 W WASHINGTON ST,EUFAULA,36027.0,Short Term,1005.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,245.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152133,291302,2020-07-31,NV,291302,WILLIAM BEE RIRIE HOSPITAL,1500 AVENUE H,ELY,89301.0,Critical Access Hospitals,32033.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,83.0,0.0
152134,290019,2020-07-31,NV,290019,CARSON TAHOE REGIONAL MEDICAL CENTER,1600 MEDICAL PARKWAY,CARSON CITY,89703.0,Short Term,32510.0,...,0.0,0.0,-999999.0,-999999.0,-999999.0,-999999.0,0.0,0.0,679.0,0.0
152135,292008,2020-07-31,NV,292008,CARSON TAHOE CONTINUING CARE HOSPITAL,775 FLEISCHMANN WAY 2ND FLR,CARSON CITY,89703.0,Long Term,32510.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
152398,331310,2020-07-31,NY,331310,ELLENVILLE REGIONAL HOSPITAL,10 HEALTHY WAY,ELLENVILLE,12428.0,Critical Access Hospitals,36111.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,234.0,0.0


In [6]:
#double check for null values
covid_df.isna().sum()

hospital_pk                                                       0
collection_week                                                   0
state                                                             0
ccn                                                               0
hospital_name                                                     0
                                                                 ..
previous_day_admission_adult_covid_suspected_80+_7_day_sum        0
previous_day_admission_adult_covid_suspected_unknown_7_day_sum    0
previous_day_admission_pediatric_covid_suspected_7_day_sum        0
previous_day_total_ED_visits_7_day_sum                            0
previous_day_admission_influenza_confirmed_7_day_sum              0
Length: 93, dtype: int64

In [7]:
#look for duplicated values
covid_df.duplicated().sum()

0

In [9]:
covid_df.describe()

,zip,fips_code,total_beds_7_day_avg,all_adult_hospital_beds_7_day_avg,all_adult_hospital_inpatient_beds_7_day_avg,inpatient_beds_used_7_day_avg,all_adult_hospital_inpatient_bed_occupied_7_day_avg,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg,total_adult_patients_hospitalized_confirmed_covid_7_day_avg,total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg,...,previous_day_admission_adult_covid_suspected_30-39_7_day_sum,previous_day_admission_adult_covid_suspected_40-49_7_day_sum,previous_day_admission_adult_covid_suspected_50-59_7_day_sum,previous_day_admission_adult_covid_suspected_60-69_7_day_sum,previous_day_admission_adult_covid_suspected_70-79_7_day_sum,previous_day_admission_adult_covid_suspected_80+_7_day_sum,previous_day_admission_adult_covid_suspected_unknown_7_day_sum,previous_day_admission_pediatric_covid_suspected_7_day_sum,previous_day_total_ED_visits_7_day_sum,previous_day_admission_influenza_confirmed_7_day_sum
count,95123.00000,95123.000000,95123.000000,95123.000000,95123.000000,95123.000000,95123.000000,95123.000000,95123.000000,95123.000000,...,95123.000000,95123.000000,95123.000000,95123.000000,95123.000000,95123.000000,95123.000000,95123.000000,95123.000000,95123.000000
mean,54037.82408,29469.899866,-655.694371,-1722.942230,-5306.865819,-76609.870024,-78939.927870,-274552.949357,-275269.579263,-118530.473890,...,-134362.251012,-147450.492163,-163471.340654,-170756.119529,-172007.142374,-169988.845369,-19805.463789,-59175.360439,-2615.074051,-41104.588848
std,26825.41861,16213.648071,28629.628626,43224.771987,73532.359332,266163.045943,269821.587169,446310.478348,446669.776777,323236.960286,...,341043.424013,354556.869619,369798.307277,376299.555744,377390.472813,375626.635413,139333.569567,235954.991293,55157.949343,198533.170854
min,603.00000,1001.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,...,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000
25%,33063.50000,17055.000000,28.100000,25.000000,23.000000,10.400000,9.400000,-999999.000000,-999999.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,58.000000,0.000000
50%,55404.00000,29037.000000,70.000000,63.700000,54.000000,34.700000,31.900000,4.900000,4.100000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,248.000000,0.000000
75%,76180.00000,42101.000000,219.000000,197.000000,167.600000,132.600000,125.300000,22.100000,19.600000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,633.000000,0.000000
max,99901.00000,78020.000000,27024.700000,27013.600000,26945.100000,26059.000000,64459.100000,559.700000,771.100000,85.300000,...,96.000000,106.000000,145.000000,190.000000,205.000000,201.000000,377.000000,407.000000,66707.000000,50.000000
